# Verbosifier

### *When more is more*

---
## Let's get it set up

- Install gpt-2-simple
- Download the model
- Start a TensorFlow session

Note that I'm using [minimaxir's gpt-2-simple package](https://github.com/minimaxir/gpt-2-simple) to make the code in this notebook a bit *less* verbose :)

In [5]:
import warnings
warnings.filterwarnings(action="ignore")

!pip install --user gpt-2-simple
import gpt_2_simple as gpt2
model_name = "124M"
gpt2.download_gpt2(model_name=model_name)

sess = gpt2.start_tf_sess()

Fetching checkpoint: 1.05Mit [00:00, 353Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 40.6Mit/s]                                                   
Fetching hparams.json: 1.05Mit [00:00, 527Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 498Mit [00:02, 210Mit/s]                                   
Fetching model.ckpt.index: 1.05Mit [00:00, 423Mit/s]                                                
Fetching model.ckpt.meta: 1.05Mit [00:00, 69.2Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 68.4Mit/s]                                                      


---

## How does GPT-2 behave out of the box?

- Finetune a model on some academic writing
- Save that model!
- Generate with a prompt, varying the temperature

In [7]:
# Finetuning on all academic writings in the COCA corpus from 1990 to 2012
gpt2.finetune(
    sess,
    "text/all_acad.txt",
    model_name=model_name,
    run_name="all_acad",
    steps=300, # Maximum number of training epochs
    print_every=5,
)

W1018 12:48:05.502055 140194596099904 deprecation.py:323] From /home/pgaddiso/.local/lib/python3.7/site-packages/gpt_2_simple/src/sample.py:17: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ResourceExhaustedError: OOM when allocating tensor with shape[50257,768] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node model/wte/Adam_1/Assign (defined at /home/pgaddiso/.local/lib/python3.7/site-packages/gpt_2_simple/src/accumulate.py:35) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Original stack trace for 'model/wte/Adam_1/Assign':
  File "/opt/conda/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/opt/conda/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/opt/conda/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/opt/conda/lib/python3.7/site-packages/ipykernel

In [9]:
import tensorflow as tf
sess.run(tf.contrib.memory_stats.MaxBytesInUse())

2359683584

In [10]:
2359683584 / 1073741824

2.1976265907287598